In [ ]:
!gmx --version

In [ ]:
%%bash

git clone https://github.com/m3g/packmol.git
cd packmol
git checkout v20.3.5
./configure
make

In [ ]:
!git clone https://github.com/zhmurov/trappeua

In [ ]:
!packmol/packmol < input/packmol.inp

In [ ]:
$GMX editconf -f conf.pdb -o conf.gro -box 4 4 4
$GMX grompp -f input/em.mdp -c conf.gro -o em.tpr
$GMX mdrun -deffnm em

$GMX grompp -f input/nvt.mdp -c em.gro -o nvt.tpr
$GMX mdrun -deffnm nvt
$GMX grompp -f input/npt.mdp -c nvt.gro -o npt.tpr
$GMX mdrun -deffnm npt